In [200]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kuet-preli-1/my_fav_recipes.txt


In [201]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [202]:
pip install langchain transformers faiss-gpu sentence-transformers torch

Note: you may need to restart the kernel to use updated packages.


In [203]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


def load_and_cache_model(model_name: str, cache_dir: str = "./model_cache"):
    """
    Load and cache the model and tokenizer locally.
    """
    print(f"Loading model '{model_name}' locally...")
    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
    model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_dir, device_map="auto")
    print(f"Model '{model_name}' loaded successfully.")
    return tokenizer, model


In [204]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_documents(documents):
    """
    Split documents into manageable chunks using LangChain's splitter.
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(documents)
    return split_docs
def clean_response(response):
    """
    Remove unrelated or redundant content from the model's response.
    """
    # Keep only the recipe section by identifying structure
    recipe_start = response.find("Name:")
    if recipe_start != -1:
        response = response[recipe_start:]

    return response.strip()


In [205]:
from langchain.schema import Document

def parse_recipes(file_path: str):
    """
    Parse recipes from the text file into LangChain Document objects.
    """
    with open(file_path, "r") as file:
        content = file.read()

    # Split recipes by "# Recipe"
    raw_recipes = content.split("# Recipe")
    documents = []

    for raw_recipe in raw_recipes:
        raw_recipe = raw_recipe.strip()
        if not raw_recipe:
            continue
        documents.append(Document(page_content=raw_recipe))
    
    return documents
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def create_vectorstore(documents):
    """
    Create a FAISS vector store for efficient retrieval.
    """
    # Use Hugging Face embeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    # Create FAISS index from documents
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore

def retrieve_relevant_documents(query, vectorstore, top_k=3):
    """
    Retrieve the most relevant documents for a given query.
    """
    docs = vectorstore.similarity_search(query, k=top_k)
    return docs
def create_rag_prompt(relevant_docs, query, max_recipes=2):
    """
    Create a prompt with the most relevant recipes for the user's query.
    """
    # Use only the top relevant recipes
    relevant_text = "\n\n".join([doc.page_content.strip() for doc in relevant_docs[:max_recipes]])
    
    # Structured prompt
    prompt = (
        f"Here are the most relevant recipes based on your query:\n\n"
        f"{relevant_text}\n\n"
        f"User Query: {query}\n\n"
        f"Provide a single complete recipe that matches the query without including irrelevant information."
    )
    return prompt



from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def generate_rag_response(prompt, tokenizer, model):
    """
    Generate a response using the RAG pipeline.
    """
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate the output
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return response


In [206]:
def query_local_llm(query, vectorstore, tokenizer, model, max_recipes=2):
    """
    Query the local LLM with retrieved recipes, ensuring concise and relevant output.
    """
    # Retrieve relevant documents
    docs = vectorstore.similarity_search(query, k=max_recipes)
    if not docs:
        return "No relevant recipes found."

    # Create a structured and focused prompt
    prompt = create_rag_prompt(docs, query)

    # Tokenize the prompt
    inputs = tokenizer(
        prompt, 
        return_tensors="pt", 
        truncation=True, 
        max_length=1024
    ).to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate the response
    outputs = model.generate(
        **inputs,
        max_new_tokens=1500,  # Limit response length
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and clean the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    return response


In [207]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub

def load_recipes(file_path: str):
    """
    Load recipes from a text file and prepare them as documents.
    """
    loader = TextLoader(file_path)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(documents)
    return split_docs


In [208]:
def create_vectorstore(documents):
    """
    Create a FAISS vector store from the recipe documents.
    """
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore


In [209]:
def create_prompt():
    """
    Create a prompt template for the LLM to suggest recipes.
    """
    return PromptTemplate(
        input_variables=["relevant_docs", "query"],
        template="Relevant recipes:\n{relevant_docs}\n\nUser Query: {query}\nAnswer:"
    )

def query_llm(query, vectorstore, llm):
    """
    Query the LLM with relevant recipes based on the user's query.
    """
    # Retrieve relevant documents
    docs = vectorstore.similarity_search(query, k=3)
    relevant_docs = "\n".join([doc.page_content for doc in docs])

    # Use LangChain LLMChain to generate a response
    prompt = create_prompt()
    chain = LLMChain(prompt=prompt, llm=llm)
    response = chain.run(relevant_docs=relevant_docs, query=query)
    return response


In [210]:
def retrieve_relevant_recipes(query, vectorstore, top_k=3):
    """
    Retrieve the most relevant recipes for a given query.
    """
    results = vectorstore.similarity_search(query, k=top_k)
    return results


In [211]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

def create_retrieval_chain(vectorstore, model_name="EleutherAI/gpt-neo-1.3B"):
    """
    Create a RetrievalQA chain using a local LLM and vectorstore.
    """
    # Load the model locally
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
    hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

    # Define the prompt template
    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template="Context:\n{context}\n\nQuestion: {question}\n\nAnswer:"
    )

    # Create RetrievalQA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=HuggingFacePipeline(pipeline=hf_pipeline),
        retriever=vectorstore.as_retriever(),
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt_template}
    )

    return qa_chain


In [212]:
def extract_first_recipe_block(response):
    """
    Extracts the content from the first '#' and terminates at the second '#',
    skipping the '#' line itself.
    """
    lines = response.splitlines()
    start_collecting = False
    collected_lines = []

    for line in lines:
        if line.startswith("#"):
            if start_collecting:  # If a second '#' is found, stop collecting
                break
            start_collecting = True  # Start collecting after the first '#'
            continue  # Skip the '#' line itself
        if start_collecting:
            collected_lines.append(line.strip())  # Collect non-empty lines

    return "\n".join(collected_lines)

In [213]:
def main():
    # File path to recipes
    file_path = "/kaggle/input/kuet-preli-1/my_fav_recipes.txt"

    # Load, split, and index recipes
    documents = load_recipes(file_path)
    split_docs = split_documents(documents)
    vectorstore = create_vectorstore(split_docs)

    # Load the model
    MODEL_NAME = "EleutherAI/gpt-neo-1.3B"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")

    # User query
    query = "Suggest a quick dinner recipe with chicken and broccoli."

    # Get response from the LLM
    response = query_local_llm(query, vectorstore, tokenizer, model)
    cleaned_response = extract_first_recipe_block(response)

    # Print the final response
    print(f"Generated Recipe:\n{cleaned_response}")


In [214]:
main()

Generated Recipe:
Name: Vegetable Stir Fry
Ingredients: Broccoli: 1 head, Bell Peppers: 2 medium, Carrots: 2 medium, Soy Sauce: 2 tbsp, Garlic: 3 cloves, Ginger: 1 inch piece, grated
Taste: Savory
Cuisine: Asian
Preparation Time: 20
Instructions:
1. Heat oil in a wok.
2. Add garlic and ginger, and stir-fry for 1 minute.
3. Add vegetables and stir-fry for 5 minutes.
4. Add soy sauce, cook for another 2 minutes, and serve.

